In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/TransportMap/")

 Activating environment at `/media/mat/HDD/TransportMap/Project.toml`


In [61]:
using Revise
using TransportMap
using Distributions
# using DocStringExtensions
using LinearAlgebra
using ProgressMeter
using Test
using BenchmarkTools
using DataInterpolations
using ProgressMeter
using Plots
using JLD
using OrdinaryDiffEq
using SpecialMatrices
using LaTeXStrings
using Colors
using NamedColors
using ShiftedArrays
using DifferentialEquations
using DifferentialEquations.EnsembleAnalysis
using StaticArrays

In [3]:
### Parallel Monte Carlo

In [4]:
function lorenz_static(u,p,t)
 dx = 10.0*(u[2]-u[1])
 dy = u[1]*(28.0-u[3]) - u[2]
 dz = u[1]*u[2] - (8/3)*u[3]
 @SVector [dx,dy,dz]
end

lorenz_static (generic function with 1 method)

In [6]:
u0 = @SVector [1.0,0.0,0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz_static,u0,tspan)
@time solve(prob,Tsit5())

  3.028968 seconds (9.08 M allocations: 482.221 MiB, 4.31% gc time)


retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 1278-element Array{Float64,1}:
   0.0
   3.5678604836301404e-5
   0.0003924646531993154
   0.0032623943767239373
   0.009058053457510118
   0.01695644120128117
   0.027689918393193867
   0.041856296355489264
   0.06024033797392694
   0.08368531582638437
   0.11336486810963423
   0.14862167169107887
   0.18703962119869624
   ⋮
  99.08175214441627
  99.16184949761713
  99.24303990603177
  99.33001063690615
  99.42833629318856
  99.51155000156021
  99.61703432215923
  99.70606422440345
  99.79129412531262
  99.86393440202542
  99.94038628938698
 100.0
u: 1278-element Array{SArray{Tuple{3},Float64,1,3},1}:
 [1.0, 0.0, 0.0]
 [0.9996434557625105, 0.0009988049817849058, 1.7814347887991892e-8]
 [0.9961045497425811, 0.010965399721242457, 2.1469553658389215e-6]
 [0.9693592810889341, 0.08977024478153006, 0.00014380067522962658]
 [0.924204512737806, 0.24228855631355772, 0.0010461574259688034]
 [0.8800457093843667, 0.4387

In [12]:
function lorenz96ode!(du,u,p,t)
    F = 8.0
    n = size(u,1)
    du[1] = (u[2]-u[end-1])*u[end] - u[1] + F
    du[2] = (u[3]-u[end])*u[1] - u[2] + F
    du[end] = (u[1] - u[end-2])*u[end-1] - u[end] + F

    @inbounds for i=3:n-1
        du[i] = (u[i+1] - u[i-2])*u[i-1] - u[i] + F
    end
end
u0 = randn(40)
Δt = 0.01
tspan = (0.0,100.0)
prob = ODEProblem(lorenz96ode!,u0,tspan)
@btime sol = solve(prob, Tsit5())

  961.111 μs (13008 allocations: 4.23 MiB)


retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 1294-element Array{Float64,1}:
   0.0
   0.012606212575817392
   0.05615178155003704
   0.12314589536908638
   0.2054891564963563
   0.3087654127953866
   0.43875319719872663
   0.5767052029459816
   0.7145596975662918
   0.8458863302797996
   0.974590668348284
   1.0828645375405392
   1.1810209523793895
   ⋮
  99.22053127469982
  99.29196484247328
  99.3679829564357
  99.43871025871057
  99.51684724678168
  99.59221452635698
  99.67476728801012
  99.74897296964157
  99.83606692575168
  99.9170044068876
  99.98703139414388
 100.0
u: 1294-element Array{Array{Float64,1},1}:
 [1.0176740329937466, -0.16210752143151916, -0.3586614781511746, 0.14635466408945016, 1.1296678527436217, -0.9565550466224526, -0.45015866296573526, -0.5547392234073181, -0.10288169328221237, -0.5088493381866488  …  -2.289648279692887, 1.4098431804296125, -0.1256898968404731, -0.08983086501880576, 0.5810315169558025, -0.7389088691268745, -0.

In [95]:
?reinit!

search: reinit! reeval_internals_due_to_modification! atreplinit



┌ Warning: /home/mat/.julia/packages/Plots/V8QVi/src/init.jl__@require__ no longer exists, deleting all methods
└ @ Revise /home/mat/.julia/packages/Revise/MgvIv/src/Revise.jl:555


```
reinit!(integrator::DEIntegrator,args...; kwargs...)
```

The reinit function lets you restart the integration at a new value.

# Arguments

  * `u0`: Value of `u` to start at. Default value is `integrator.sol.prob.u0`

# Keyword Arguments

  * `t0`: Starting timepoint. Default value is `integrator.sol.prob.tspan[1]`
  * `tf`: Ending timepoint. Default value is `integrator.sol.prob.tspan[2]`
  * `erase_sol=true`: Whether to start with no other values in the solution, or keep the previous solution.
  * `tstops`, `d_discontinuities`, & `saveat`: Cache where these are stored. Default is the original cache.
  * `reset_dt`: Set whether to reset the current value of `dt` using the automatic `dt` determination algorithm. Default is `(integrator.dtcache == zero(integrator.dt)) && integrator.opts.adaptive`
  * `reinit_callbacks`: Set whether to run the callback initializations again (and `initialize_save` is for that). Default is `true`.
  * `reinit_cache`: Set whether to re-run the cache initialization function (i.e. resetting FSAL, not allocating vectors) which should usually be true for correctness. Default is `true`.

Additionally, once can access [`auto_dt_reset!`](@ref) which will run the auto `dt` initialization algorithm.

---

```
reinit!(integrator::DDEIntegrator[, u0 = integrator.sol.prob.u0;
        t0 = integrator.sol.prob.tspan[1],
        tf = integrator.sol.prob.tspan[2],
        erase_sol = true,
        kwargs...])
```

Reinitialize `integrator` with (optionally) different initial state `u0`, different integration interval from `t0` to `tf`, and erased solution if `erase_sol = true`.


In [89]:
function lorenz96staticode!(du,u,p,t)
    F = 8.0
    n = size(u,1)
    du[1] = (u[2]-u[end-1])*u[end] - u[1] + F
    du[2] = (u[3]-u[end])*u[1] - u[2] + F
    du[end] = (u[1] - u[end-2])*u[end-1] - u[end] + F

    @inbounds for i=3:n-1
        du[i] = (u[i+1] - u[i-2])*u[i-1] - u[i] + F
    end
    return du
end
u0 = randn(40)
Δt = 0.01
tspan = (0.0,100.0)
prob = ODEProblem(lorenz96staticode!,x0,tspan)
sol = solve(prob, Tsit5())

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 1275-element Array{Float64,1}:
   0.0
   0.012306217806358311
   0.05630456952914854
   0.12582286017948907
   0.21186497833798917
   0.3237344554974292
   0.4550002855497027
   0.5867230903981321
   0.720093207368685
   0.8482610989295511
   0.9661017990310082
   1.0759970427705694
   1.1590277622185468
   ⋮
  99.15760267711296
  99.2329369612347
  99.29837211144805
  99.37624444952228
  99.45023995671913
  99.52922185431561
  99.61507651330443
  99.69709977234005
  99.77839217954966
  99.85885853941011
  99.94490461813895
 100.0
u: 1275-element Array{Array{Float64,1},1}:
 [-0.6315379515215261, -0.4592930800057147, -0.8503872608845433, 1.0915831210346894, -1.2816217461530213, -0.35075121910474544, -0.16075316249656907, -1.5101366713585729, 1.7727980243991714, -0.3907660704193865  …  0.7591253592124714, 0.39784451859706677, -1.037770746131684, -0.3046110858753022, -0.5884975768614219, -0.9002221963113536, 0.8

In [90]:
sol.u[end]

40-element Array{Float64,1}:
  8.256806847749964
  0.5126773419640843
 -1.3357310385791523
  3.418700585232726
  5.614667631254987
  1.0765072514284388
  0.5905795908393764
 -2.0182458905673073
  1.2800737938570712
  3.028037635386499
  7.348123249042639
 11.486287793082772
  7.29827552781238
  ⋮
 -1.2485072396934775
 -4.115986256623816
  4.044531562232652
  7.950865142027807
 -0.054962449761738016
  4.569029086892615
  8.720785562127475
 -0.05836943148915462
  4.490255474534123
  7.026442416791662
  5.868634393743815
  7.472116574375814

In [91]:
sim[1].u[2]

40-element Array{Float64,1}:
  8.256806847749964
  0.5126773419640843
 -1.3357310385791523
  3.418700585232726
  5.614667631254987
  1.0765072514284388
  0.5905795908393764
 -2.0182458905673073
  1.2800737938570712
  3.028037635386499
  7.348123249042639
 11.486287793082772
  7.29827552781238
  ⋮
 -1.2485072396934775
 -4.115986256623816
  4.044531562232652
  7.950865142027807
 -0.054962449761738016
  4.569029086892615
  8.720785562127475
 -0.05836943148915462
  4.490255474534123
  7.026442416791662
  5.868634393743815
  7.472116574375814

In [83]:
ens.S[:,1]

40-element Array{Float64,1}:
  3.827810457116878
  3.3613310256218196
  0.5914635058991771
  2.2567001876430326
  5.350162323331959
  6.569701605753321
 -3.7996438992937436
  2.270656636315519
  3.7825988402332635
  8.0051677994196
  0.6006946712499444
  0.8914349429656017
  5.382954072126129
  ⋮
  2.2726880553283664
  0.39715961856621496
  2.8877465942579454
  6.263462394051014
  5.460786299588545
 -4.1202267972536015
  1.045969356138223
  3.2418587508109553
 12.787803042053367
  0.9047224274488211
 -1.1107986474958784
  3.5056336849360634

In [30]:
# function prob_func(prob,i,repeat)
#     ODEProblem(prob.f,rand()*prob.u0,prob.tspan)
# end

In [56]:
struct MyStruct
    S::Array{Float64,2} 
end

In [84]:
Ne = 100
ens = MyStruct(randn(40,Ne))

MyStruct([-0.6315379515215261 -1.3450468646173432 … 1.0547936257684565 -0.6670934482297195; -0.4592930800057147 -0.26734101464038523 … 1.2299471776279942 -0.8965114837645832; … ; -0.13665867585768257 -0.6315141445006772 … -2.417359311116126 -0.6212741137164657; -0.2859610386466935 -0.05466855506639558 … -1.6049969507794692 -0.1591783223646082])

In [85]:
x0 = ens.S[:,1]

40-element Array{Float64,1}:
 -0.6315379515215261
 -0.4592930800057147
 -0.8503872608845433
  1.0915831210346894
 -1.2816217461530213
 -0.35075121910474544
 -0.16075316249656907
 -1.5101366713585729
  1.7727980243991714
 -0.3907660704193865
  1.420157978645884
  0.878417689146963
 -0.14433838919292852
  ⋮
 -0.720023754042311
 -1.1503813503692601
  0.7591253592124714
  0.39784451859706677
 -1.037770746131684
 -0.3046110858753022
 -0.5884975768614219
 -0.9002221963113536
  0.8200710295449039
  0.6518354143702281
 -0.13665867585768257
 -0.2859610386466935

In [86]:
ens.S

40×100 Array{Float64,2}:
 -0.631538  -1.34505     1.23096    …   0.154879     1.05479   -0.667093
 -0.459293  -0.267341   -0.670663      -0.571877     1.22995   -0.896511
 -0.850387  -0.0128701  -0.669741      -1.98951      1.56041    1.20668
  1.09158    0.306034   -0.471196      -2.06327     -0.646434  -1.60931
 -1.28162    0.211606   -0.541686      -0.113015     0.856043   0.76717
 -0.350751   0.860896    0.83744    …   0.542441    -1.49973    0.0321804
 -0.160753  -0.911187   -1.24517       -0.61452     -0.242997   0.763819
 -1.51014    0.266547   -0.134852      -1.19217      1.35759   -0.658778
  1.7728    -0.715224    0.111107       0.0229144    0.974136   0.414963
 -0.390766   1.34584     0.203745      -0.574685     1.62798   -0.539553
  1.42016    1.30027     0.230546   …  -0.0610482   -0.18006    1.1934
  0.878418   0.751215    0.258144       2.05639     -0.690114  -0.19373
 -0.144338   0.653869   -0.0533939      1.26344     -0.86101   -0.0339906
  ⋮                           

In [87]:
function prob_func(prob,i,repeat)
    ODEProblem(prob.f,view(ens.S,:,i),prob.tspan)
end

prob_func (generic function with 1 method)

In [93]:
ensemble_prob = EnsembleProblem(prob,output_func = (sol,i) -> (sol[end], false), prob_func=prob_func)
sim = solve(ensemble_prob, Tsit5(), EnsembleThreads(),trajectories=Ne, dense = false, save_everystep=false);

In [77]:
@time for i=1:Ne
    ens.S[:,i] .= sim[i].u[2]
end

  0.000174 seconds (601 allocations: 15.656 KiB)


In [94]:
sim[1]

40-element Array{Float64,1}:
  8.256806847749964
  0.5126773419640843
 -1.3357310385791523
  3.418700585232726
  5.614667631254987
  1.0765072514284388
  0.5905795908393764
 -2.0182458905673073
  1.2800737938570712
  3.028037635386499
  7.348123249042639
 11.486287793082772
  7.29827552781238
  ⋮
 -1.2485072396934775
 -4.115986256623816
  4.044531562232652
  7.950865142027807
 -0.054962449761738016
  4.569029086892615
  8.720785562127475
 -0.05836943148915462
  4.490255474534123
  7.026442416791662
  5.868634393743815
  7.472116574375814

In [21]:
rk = RK(zeros(40), 0.01, lorenz96(zeros(40)), rk=TransportMap.RK4)
f(t,x) = rk(t,x)

f (generic function with 1 method)

In [22]:
# rk = RK(zeros(40), 0.01, lorenz96, rk=TransportMap.RK4)
# f(t,x) = rk(t,x)

In [23]:
function prop()
    @btime begin f(0.0, randn(40)) end
end

prop (generic function with 1 method)

In [24]:
# The function has to be defined
function lorenz!(t::Float64, u, du)
    F = 8.0
    n = size(u,1)
    du[1] = (u[2]-u[end-1])*u[end] - u[1] + F
    du[2] = (u[3]-u[end])*u[1] - u[2] + F
    du[end] = (u[1] - u[end-2])*u[end-1] - u[end] + F

    @inbounds for i=3:n-1
        du[i] = (u[i+1] - u[i-2])*u[i-1] - u[i] + F
    end

return du
end

# The function has to be defined
lorenz(du) = (t,u) -> lorenz!(t, u, du)

lorenz (generic function with 1 method)

In [27]:
u = collect(1:1.0:40.0)
@btime circshift(u,-1)

  75.558 ns (1 allocation: 400 bytes)


40-element Array{Float64,1}:
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0
 11.0
 12.0
 13.0
 14.0
  ⋮
 30.0
 31.0
 32.0
 33.0
 34.0
 35.0
 36.0
 37.0
 38.0
 39.0
 40.0
  1.0

In [11]:
function lorenzcirc!(t::Float64, u, du)
    F = 8.0
    up1 = ShiftedArrays.circshift(u, (-1,0))
    um1 = ShiftedArrays.circshift(u, (1,0))
    um2 = ShiftedArrays.circshift(u, (2,0))
    du .= (up1 - um2) .* um1 - u .+ F 
    return du
end

lorenzcirc(du) = (t,u) -> lorenzcirc!(t, u, du)

lorenzcirc (generic function with 1 method)

In [12]:
function rhs()
@btime x = randn(40,200)
    
@btime begin
x = randn(40,200)
for j=1:50
for i=1:200         
    lorenz(zeros(40))(0.0, view(x,:,i))
end
        end
end
end

rhs (generic function with 1 method)

In [13]:
function rhscirc()
@btime x = randn(40,200)
@btime begin
        for  j=1:50
            lorenzcirc(zeros(40,200))(0.0, randn(40,200))
        end
    end
end

rhscirc (generic function with 1 method)

In [35]:
function lorenzode2!(du,u,p,t)
    F = 8.0
    n = size(u,1)
    du[1] = (u[2]-u[end-1])*u[end] - u[1] + F
    du[2] = (u[3]-u[end])*u[1] - u[2] + F
    du[end] = (u[1] - u[end-2])*u[end-1] - u[end] + F

    @inbounds for i=3:n-1
        du[i] = (u[i+1] - u[i-2])*u[i-1] - u[i] + F
    end
end

lorenzode2! (generic function with 1 method)

In [38]:
u0 = randn(40)
tspan = (0.0,100.0)
prob = ODEProblem(lorenzode!,u0,tspan)
@btime sol = solve(prob, dt = 0.01, adaptive = false, RK4())

  44.189 ms (530072 allocations: 75.01 MiB)


retcode: Success
Interpolation: 3rd order Hermite
t: 10001-element Array{Float64,1}:
   0.0
   0.01
   0.02
   0.03
   0.04
   0.05
   0.060000000000000005
   0.07
   0.08
   0.09
   0.09999999999999999
   0.10999999999999999
   0.11999999999999998
   ⋮
  99.8900000000142
  99.9000000000142
  99.91000000001421
  99.92000000001421
  99.93000000001422
  99.94000000001422
  99.95000000001423
  99.96000000001423
  99.97000000001424
  99.98000000001424
  99.99000000001425
 100.0
u: 10001-element Array{Array{Float64,1},1}:
 [0.658807627866634, -1.8300076823361142, -1.3836426871446557, -0.1682449005498444, -0.5623453341804536, -0.5072083307934195, -0.01930915070704292, 0.9478239056709882, 1.3238757920530277, -0.8123211786591209  …  1.0170816410402754, 1.0171433041879885, 0.8269190783695535, -0.9471504814208375, -1.0603818292518798, -0.08155005964715464, -1.3258661352863272, -0.9014860756888204, -0.759705263733499, -1.330362186761284]
 [0.7455206620878831, -1.7324740733910557, -1.2754181344503

In [45]:
function speed()
@btime begin
    u0 = randn(40)
    tspan = (0.0,100.0)
    prob = ODEProblem(lorenzode!,u0,tspan)
end

@btime begin
    u0 = randn(40)
    tspan = (0.0,100.0)
    prob = ODEProblem(lorenzode!,u0,tspan)
    sol = solve(prob, dt = 0.01, adaptive = false, RK4())
end
end

speed (generic function with 1 method)

In [46]:
speed()

  29.851 μs (296 allocations: 23.83 KiB)
  44.821 ms (530368 allocations: 75.03 MiB)


retcode: Success
Interpolation: 3rd order Hermite
t: 10001-element Array{Float64,1}:
   0.0
   0.01
   0.02
   0.03
   0.04
   0.05
   0.060000000000000005
   0.07
   0.08
   0.09
   0.09999999999999999
   0.10999999999999999
   0.11999999999999998
   ⋮
  99.8900000000142
  99.9000000000142
  99.91000000001421
  99.92000000001421
  99.93000000001422
  99.94000000001422
  99.95000000001423
  99.96000000001423
  99.97000000001424
  99.98000000001424
  99.99000000001425
 100.0
u: 10001-element Array{Array{Float64,1},1}:
 [0.051013039501466684, 0.5458528581624496, 1.5331777159258833, 0.8716435666085733, 0.17437800511214838, 0.8998092475361593, -1.0973538031087693, -1.2366499964222573, 1.0058838561286503, 0.6514281893955682  …  0.1735298868335711, -0.16083961146707498, 0.03407059897514544, 1.6148313367946017, 0.6387977517847352, -0.6353027018086681, -1.7664841069082855, -0.6252262006570815, 0.747242566753497, 0.38321821334658523]
 [0.12929185735472157, 0.6210497990090073, 1.602249143695088,

In [49]:
44.821e-3/10000

4.4821e-6

In [50]:
4.4821e-6*4000*60

1.075704